<a href="https://colab.research.google.com/github/mchlltene/Ensemble_Learning/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Import Python Packages**



In [1]:
##########################################################
# 1. IMPORT ALL PACKAGES
##########################################################
import pandas as pd
from sklearn.metrics import confusion_matrix
import math
from collections import Counter
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
import numpy as np
import string
from sklearn import svm
from sklearn.tree import export_graphviz
import re
import nltk
nltk.download("stopwords"); # Filtering (Stopword Removal)
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# import pydot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import multilabel_confusion_matrix
from statistics import mean 

import joblib

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#PACKAGE UNTUK CONNECT G-DRIVE N G-COLLAB 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# #PANDAS LIBRARY UNTUK DATAFRAME
# import pandas as pd
# dataset = ('/content/drive/MyDrive/DataScrap/test_data.csv')
# df_review = pd.read_csv(dataset, sep=';')
# df_review.head()


In [4]:
##########################################################
# 2. LOAD TESTING AND TRAINING DATASET
# header 0 means the first row is name of the coloumn 
##########################################################
dateset_train =("/content/drive/MyDrive/64373/DATA/DATA_TRAINING.csv")
dateset_test = ("/content/drive/MyDrive/64373/DATA/DATA_TESTINGG.csv")

# dateset_train =("/content/training.csv")
# dateset_test = ("/content/test.csv")


df_train = pd.read_csv(dateset_train , sep=';')
df_test = pd.read_csv(dateset_test, sep=';')

df_train = df_train.dropna(subset=['content']) # Drop rows if missing values
df_test = df_test.dropna(subset=['content']) # Drop rows if missing values

# Check all classes/labels in training data
all_label = set(df_train['score'].tolist())

# Data distribution for each score
dst_train = Counter(df_train['score'])

# View sample data
df_train

,score,label,at,content
0,1,Negatif,17/06/2022,aplikasi loadingnya lama struk belanja tdk re...
1,1,Negatif,17/06/2022,aplikasi yg sangat bermanfat
2,1,Negatif,17/06/2022,dulu deh mbaknya g nanya member ada apa gag p...
3,1,Negatif,17/06/2022,kali pemesanan kecewa yg pesan cod di kirim h...
4,1,Negatif,17/06/2022,kali pesan dan selalu di cancel setidak nya k...
...,...,...,...,...
8995,0,Positif,22/04/2023,Woww.. bagus banget
8996,0,Positif,09/11/2023,Wowwww Aplikasi nya bagus banget aku suka seka...
8997,0,Positif,21/02/2023,Ya Like Ok
8998,0,Positif,23/01/2023,Yang pasti sih udah love lah ya sama alfa dari...


In [5]:
# Function to remove punctuations
def remove_punctuations(text):
  for sym_punc in string.punctuation:
    text = text.replace(sym_punc, ' ') 
    text = re.sub(' +', ' ', text) # replace multiple spaces with single space
  return text;

# Function to remove duplicate characters
def remove_duplicate_chars(text):
  pattern = r'(.)\1+' # (.) any character repeated (\+) more than
  repl = r'\1'        # replace it once
  text = re.sub(pattern, repl, text);
  return text;

def normalization(text):
  return text.replace("maf", "maaf").replace("ko", "kenapa").replace("kok", "kenapa").replace("g", "tidak").replace("pdhl", "padahal").replace("in", "ini").replace("bnget","banget").replace("blnja","belanja").replace("ga","tidak").replace("tp","tapi").replace("pdhal","padahal").replace("utk","untuk").replace("sy","saya").replace("tdk","tidak").replace("apk","aplikasi").replace("ngak","tidak").replace("dlu","dulu").replace("gak","tidak").replace("klu","kalau").replace("tlg","tolong").replace("gak","tidak").replace("adain","adakan").replace("jd","jadi").replace("skrng","sekarang").replace("bgus","bagus").replace("nerima","terima").replace("kenpa","kenapa").replace("uda","sudah").replace("skrg","sekarang").replace("mnggu","menunggu").replace("krja","kerja").replace("plg","pulang").replace("happy","senang").replace("good","bagus").replace("msuk","masuk").replace("pd","pada").replace("aja","saja").replace("bgaus","bagus").replace("klau","kalau").replace("jd","jadi").replace("sdh","sudah").replace("sdg","sedang").replace("donload","download").replace("udah","sudah").replace("lamaaaaa","lama").replace("klo","kalau").replace("lncar","lancar").replace("ngirim","kirim").replace("banyk","banyak").replace("smg","semoga").replace("blanja","belanja").replace("ngga","tidak").replace("lgsg","langsung").replace("gada","tidak ada").replace("pgn","ingin").replace("gada","tidak ada").replace("ktawa","tertawa").replace("aq","aku").replace("no","nomor").replace("ok","oke").replace("oky","oke").replace("oklah","oke").replace("lemottt","lemot").replace("liat","lihat").replace("bs","bisa").replace("eror","error").replace("ngk","tidak").replace("yg","yang").replace("apk","aplikasi").replace("soalx","soalnya").replace("sbg","sebagai").replace("gw","saya").replace("pda","pada").replace("vouche","vocher").replace("mlh","malah").replace("slah","salah").replace("slah","salah").replace("susa","susah").replace("yh","yah").replace("usefull","membantu").replace("belnjar","belanja").replace("sippp","sip").replace("akuh","saya").replace("alfagiftttt","alfagift").replace("tq","terima kasih").replace("kt","kata").replace("bguuuus","bagus").replace("helpfull","membantu")

# Data preprocessingk
df_train['content'] = df_train['content'].str.replace('\d+', '', regex=True) # Remove numbers from string
df_train['content'] = df_train['content'].apply(lambda x:x.lower()) # Change Strings to lowercase
df_train["content"] = df_train['content'].apply(remove_punctuations) # Remove punctuations (Annotation Removal)
df_train["content"] = df_train['content'].apply(remove_duplicate_chars) # Remove repeated characters
df_train['content'] = df_train['content'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii')) # Remove emojis
# df_train['content'] = df_train['content'].apply(normalization) 

#TEST DATA
# Data preprocessing
df_test['content'] = df_test['content'].str.replace('\d+', '', regex=True) # Remove numbers from string
df_test['content'] = df_test['content'].apply(lambda x:x.lower()) # Change Strings to lowercase
df_test["content"] = df_test['content'].apply(remove_punctuations) # Remove punctuations (Annotation Removal)
df_test["content"] = df_test['content'].apply(remove_duplicate_chars) # Remove repeated characters
df_test['content'] = df_test['content'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii')) # Remove emojis
# df_test['content'] = df_test['content'].apply(normalization) 

df_train.head(10)


,score,label,at,content
0,1,Negatif,17/06/2022,aplikasi loadingnya lama struk belanja tdk re...
1,1,Negatif,17/06/2022,aplikasi yg sangat bermanfat
2,1,Negatif,17/06/2022,dulu deh mbaknya g nanya member ada apa gag p...
3,1,Negatif,17/06/2022,kali pemesanan kecewa yg pesan cod di kirim h...
4,1,Negatif,17/06/2022,kali pesan dan selalu di cancel setidak nya k...
5,1,Negatif,17/06/2022,kali transaksi tidak masuk alfagift padahal d...
6,1,Negatif,17/06/2022,knp skrg suka nyasar ke toko yg lbih jauh pad...
7,1,Negatif,17/06/2022,parah produk yang dicari kosong terus kalopun...
8,1,Negatif,17/06/2022,respon aplikasi begitu lambat bahkan lebih la...
9,1,Negatif,17/06/2022,sy kecewa bulan yg lalu pesan ambil ditempat ...


In [6]:
from warnings import filterwarnings
# Function to load filter to list (berisi kata-kata yang akan di filter)
def load_indonesia_filter():
  # get stop words
  filtering = stopwords.words("indonesian");
  # append additional stopword
  filtering.extend(["yg","untuk", "dg", "rt", "dgn", "ny", "d", 'klo', 'deh','sekarang','ku','jam','rumah','kan','nya','member','alfa','gift',"aplikasi",
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'tokonya', 'toko', 'bnget','ini',"dulu","kalau","jadi","sekarang","kerja","pulang","pada","sedang","download","langsung","ingin","tertawa",
                       "nomor","lihat","aplikasi","sebagai","saya","voucher","malah","yah","alfagift",'untuk', 'ga', 'krn', 'nya', 'nih', 'sih', 'emg', 'kk',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'yaaa', 'ya','nomor', 'aktif','titik','jd', 'jgn', 'sdh', 'aja', 'ka', 't', 'user','versi','nyg', 'hehe', 'yaa', 'lihat', 'nan', 'loh', 'rs','telepon','x',
                       '&amp', 'yah', 'covid','dirawat','aplikasi','apk','aplikasinya','sehari', 'kasir',"ponta","saat","atau", "anak", "pencet","login", "di", "hp", "masa", "nomor","trus","nya","masa","struk",
                       "qt", "bayar","kan","keranjang","checkout","dr","sistem","alfagift","aplikasi","alfa","kaya","baru","apk","down","hbs","jam","saat","di","app",
                      "nya","ke","sih","alfa","youtube","nafsu","belanja","kali","barang",'ambil','pas','dateng','kurir','outlet','vmg','bekasi','utara','dah','scan','tapi','butuh','besok','sat','memesan', 
                      'pake','belanjan','keranjang','edit','pokok','mending','memory','recipt','dihapus','sya','blanja','dpt','points',"point", 'kebijakanya','pelangan','dimana','foucher','cashback','klaim','sedah',
                      'alfamart','faktur','dint','nt', 'daftar','nyampe','tu','ada','beli','es','krim','carikan','leleh','kupon','min','order','memenuhi','syarat','eh','produk','produknya','kaya','pengiriman','list','brp','alamat',
                    'yg','transaksi','deh','respon','nya','checkout','pelayanan','pelayanannya'])
  filtering.remove("padahal") 
  filtering.remove('ada')
  filtering.remove('agak') 
  filtering.remove('tapi') 
  filtering.remove('juga') 
  filtering.remove('seharusnya') 
  filtering.remove('bisa') 
  filtering.remove('kenapa') 
  filtering.remove('tambah') 
  filtering.remove('sangat') 
  return filtering;

# load filter words
filter_words = load_indonesia_filter();

# remove stop words (filtering)
df_train['content'] = df_train['content'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in filter_words))
df_train = df_train.dropna(subset=['content']) # Remove Missing Values
df_train.head()

# remove stop words (filtering)
df_test['content'] = df_test['content'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in filter_words))
df_test = df_test.dropna(subset=['content']) # Remove Missing Values
df_test.head(10)

,score,label,at,content,Unnamed: 4
0,1,Negatif,27/05/2022,diperbaiki loading banget yng dicari ngk deliv...,NaN
1,1,Negatif,27/05/2022,kenapa susah dibuka udh sebulan gk bisa pke al...,NaN
2,1,Negatif,27/05/2022,rokok hilang,NaN
3,1,Negatif,27/05/2022,sebener bagus tp skrg minus bsa rokok delivery,NaN
4,1,Negatif,26/05/2022,tukar poin gak bisa udah instal berulang kx,NaN
5,1,Negatif,26/05/2022,coba fitur chat kadang pesen diantar kadang ba...,NaN
6,1,Negatif,26/05/2022,harga beda bh swety xl bh sgm gr vanila bh por...,NaN
7,1,Negatif,26/05/2022,vocher via gak nominal voucherny tertera fikti...,NaN
8,1,Negatif,26/05/2022,sinyal udah bagus kecepatan juga bagus muter m...,NaN
9,1,Negatif,26/05/2022,test pesan sore ditguin smpe mlm blm juga pesa...,NaN


In [7]:

##########################################################
# 3. SAVE CORPUS DATA AS A FILE
##########################################################

# load corpus data / kumpulan teks / dictionary (corpus data is used to generate a dictionary)
corpus_data = df_train.append(df_test) # from testing and training data
corpus = corpus_data['content'].to_numpy()


# 1-gram
model_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 1))
model_vectorizer.fit(corpus)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", model_vectorizer.vocabulary_)

# saving python object as pkl (vocabulary)
joblib.dump(model_vectorizer, "/content/drive/MyDrive/64373/Modelss/Vectorizer_Ensemble_three_class.pkl")



Vocabulary:  {'loadingnya': 4205, 'real': 6549, 'time': 7937, 'masuknya': 4430, 'sangat': 6808, 'bermanfat': 806, 'mbaknya': 4463, 'nanya': 5196, 'gag': 2528, 'padahal': 5686, 'blanjaku': 965, 'dapet': 1485, 'poin': 6314, 'alfagif': 158, 'pemesanan': 5922, 'kecewa': 3460, 'pesan': 6175, 'cod': 1351, 'kirim': 3765, 'pembayaran': 5887, 'transfer': 8056, 'jm': 3227, 'ndk': 5216, 'niat': 5410, 'delivery': 1543, 'servis': 7116, 'buka': 1126, 'nungu': 5492, 'konfirmasi': 3865, 'cancel': 1201, 'setidak': 7138, 'pemberitahuan': 5902, 'gak': 2540, 'nunguin': 5493, 'masuk': 4425, 'udah': 8182, 'afa': 39, 'id': 2981, 'nama': 5182, 'knp': 3816, 'skrg': 7263, 'suka': 7488, 'nyasar': 5527, 'lbih': 4113, 'sesuai': 7120, 'kecamatan': 3456, 'mlh': 5008, 'bngt': 1006, 'diluar': 1859, 'parah': 5765, 'dicari': 1706, 'kosong': 3896, 'kalopun': 3311, 'pasang': 5776, 'datangi': 1501, 'terteranya': 7863, 'stock': 7440, 'tersedia': 7855, 'lainya': 4049, 'minyak': 4976, 'goreng': 2717, 'gambar': 2557, 'harga': 

['/content/drive/MyDrive/64373/Modelss/Vectorizer_Ensemble_three_class.pkl']

In [8]:
##########################################################
# 4. FEATURE EXTRACTION TECHNIQUES
##########################################################
# load object vectorizer (vocabulary)
vectorizer = joblib.load("/content/drive/MyDrive/64373/Modelss/Vectorizer_Ensemble_three_class.pkl")

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)


Vocabulary:  {'loadingnya': 4205, 'real': 6549, 'time': 7937, 'masuknya': 4430, 'sangat': 6808, 'bermanfat': 806, 'mbaknya': 4463, 'nanya': 5196, 'gag': 2528, 'padahal': 5686, 'blanjaku': 965, 'dapet': 1485, 'poin': 6314, 'alfagif': 158, 'pemesanan': 5922, 'kecewa': 3460, 'pesan': 6175, 'cod': 1351, 'kirim': 3765, 'pembayaran': 5887, 'transfer': 8056, 'jm': 3227, 'ndk': 5216, 'niat': 5410, 'delivery': 1543, 'servis': 7116, 'buka': 1126, 'nungu': 5492, 'konfirmasi': 3865, 'cancel': 1201, 'setidak': 7138, 'pemberitahuan': 5902, 'gak': 2540, 'nunguin': 5493, 'masuk': 4425, 'udah': 8182, 'afa': 39, 'id': 2981, 'nama': 5182, 'knp': 3816, 'skrg': 7263, 'suka': 7488, 'nyasar': 5527, 'lbih': 4113, 'sesuai': 7120, 'kecamatan': 3456, 'mlh': 5008, 'bngt': 1006, 'diluar': 1859, 'parah': 5765, 'dicari': 1706, 'kosong': 3896, 'kalopun': 3311, 'pasang': 5776, 'datangi': 1501, 'terteranya': 7863, 'stock': 7440, 'tersedia': 7855, 'lainya': 4049, 'minyak': 4976, 'goreng': 2717, 'gambar': 2557, 'harga': 

In [9]:
# Generate feature [test and train] from dictionary
# Testing data
test_data = vectorizer.transform(df_test['content']).toarray()
feature_test = pd.DataFrame(data = test_data, columns = vectorizer.get_feature_names_out())
#df_test.to_csv('d:/test_data.csv', encoding='utf-8', index=False)

# training data
train_data = vectorizer.transform(df_train['content']).toarray()
feature_train = pd.DataFrame(data = train_data, columns = vectorizer.get_feature_names_out())

feature_test

,ab,abal,abang,abangnya,abil,abis,abisan,abs,absen,abu,...,yutub,yuzu,za,zaman,zayanious,zom,zonepun,zonk,zu,zuper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
##########################################################
# 4. ADD CLASS TO TESTING AND TRAINING DATA
##########################################################
# ['POSITIF','NEGATIF', 'NETRAL'], [0, 1, 2]

# train
class_train = df_train['score'];
feature_train['score'] = class_train

# test
class_test = df_test['score'];
feature_test['score'] = class_test

feature_test = feature_test.dropna() # Drop rows if missing values
feature_train = feature_train.dropna() # Drop rows if missing values

feature_test

,ab,abal,abang,abangnya,abil,abis,abisan,abs,absen,abu,...,yutub,yuzu,za,zaman,zayanious,zom,zonepun,zonk,zu,zuper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
##########################################################
# 6. DEFINE X AND Y VARIABLES
##########################################################
# # Testing data
x_test = feature_test.iloc[:, :-1]
y_test = feature_test['score'].tolist()

# Training data
x_train = feature_train.iloc[:, :-1]
y_train = feature_train['score'].tolist()

# Print shape of the Datasets
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
# print(feature_test.shape[0], 'test samples')

x_train shape: (9000, 8485)
9000 train samples


In [12]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [13]:
# DEKLARASI VARIABEL ESTIMATOR SEBAGAI PENAMPUNG SUB MODEL 3 ALGORITMA
estimators = []

# MEMBUAT SUB MODEL MENGGUNAKAN 3 ALGORITMA
# MODEL MENGGUNAKAN KNN
model1 = KNeighborsClassifier()
model1.fit(x_train, y_train)
pred1 = model1.predict(x_test)
estimators.append(('KNN', model1))

# MODEL MENGGUNAKAN NAIVE BAYES
model2 = RandomForestClassifier()
model2.fit(x_train, y_train)
pred2 = model2.predict(x_test)
estimators.append(('Random Forest', model2))


# MODEL MENGGUNAKAN SVM
svm = LinearSVC()
model3 = CalibratedClassifierCV(svm) 
model3.fit(x_train, y_train)
pred3 = model3.predict(x_test)
estimators.append(('SVM', model3))

In [14]:
from sklearn.ensemble import VotingClassifier
# MODEL ENSEMBLE DENGAN METODE VOTING
ensemble = VotingClassifier(estimators, voting="soft")
ensemble.fit(x_train, y_train)
pred4 = ensemble.predict(x_test)

# MENAMPILKAN PREDIKSI KEDALAM DATA FRAME
cols = {
    'Y True' : y_test,
    'Pred KNN' : pred1,
    'Random Forest' : pred2,
    'Pred SVM' : pred3,
    'Voting' : pred4

}
print("Prediksi pada setiap Algoritma")
pd.DataFrame(cols)

Prediksi pada setiap Algoritma


,Y True,Pred KNN,Random Forest,Pred SVM,Voting
0,1,1,1,1,1
1,1,1,2,1,1
2,1,1,1,1,1
3,1,1,2,1,1
4,1,1,1,1,1
...,...,...,...,...,...
995,0,0,0,0,0
996,0,0,0,0,0
997,0,0,0,0,0
998,0,0,0,0,0


In [15]:

##########################################################
# 6. SAVE MODEL
##########################################################
import pickle
# save the model to disk


filename = '/content/drive/MyDrive/64373/Modelss/Ensemble_classifier_three_class.model'
pickle.dump(ensemble, open(filename, 'wb'))


In [16]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model

VotingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                             ('Random Forest', RandomForestClassifier()),
                             ('SVM',
                              CalibratedClassifierCV(base_estimator=LinearSVC()))],
                 voting='soft')

In [17]:
##########################################################
# 7. APPLY THE TRAINED LEARNER TO TEST NEW DATA
##########################################################
# # Apply the trained perceptron to make prediction of test data
y_pred = ensemble.predict(x_test)


##########################################################
# 8. CONFUSION MATRIX
##########################################################
# Actual and predicted classes
lst_actual_class = y_test
lst_predicted_class = pred4

# label ['POSITIF', 'NETRAL','NEGATIF'], [0, 1, 2]
lst_classes = [0, 1, 2]

# Compute multi-class confusion matrix
arr_out_matrix = multilabel_confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes)

# Temp store results
store_sens = [];
store_spec = [];
store_acc = [];
store_bal_acc = [];
store_prec = [];
store_fscore = [];
store_mcc = [];

# Loop for each taget label
for no_class in range(len(lst_classes)):
    arr_data = arr_out_matrix[no_class];
    print("Predicted Performance of Digit Label/Class: {0}".format(no_class));

    tp = arr_data[1][1]
    fp = arr_data[0][1]
    tn = arr_data[0][0]
    fn = arr_data[1][0]

    sensitivity = round(tp/(tp+fn), 3);
    specificity = round(tn/(tn+fp), 3);
    accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
    balanced_accuracy = round((sensitivity+specificity)/2, 3);
    precision = round(tp/(tp + fp), 3)
    fscore = round((2 * ((precision * sensitivity) / (precision + sensitivity))), 3)
    mcc = round((((tp * tn)-(fp * fn))/ math.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))),3)

    store_sens.append(sensitivity);
    store_spec.append(specificity);
    store_acc.append(accuracy);
    store_bal_acc.append(balanced_accuracy);
    store_prec.append(precision);
    store_fscore.append(fscore);
    store_mcc.append(mcc);
    
    print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
    print("Sensitivity: {0}".format(sensitivity));
    print("Specificity: {0}".format(specificity));
    print("Accuracy: {0}".format(accuracy));
    print("Balanced Accuracy: {0}".format(balanced_accuracy));
    print("Precision: {0}".format(precision));
    print("F1-Score: {0}".format(fscore))
    print("MCC: {0}\n".format(mcc))

Predicted Performance of Digit Label/Class: 0
TP=300, FP=6, TN=694, FN=0
Sensitivity: 1.0
Specificity: 0.991
Accuracy: 0.994
Balanced Accuracy: 0.996
Precision: 0.98
F1-Score: 0.99
MCC: 0.986

Predicted Performance of Digit Label/Class: 1
TP=294, FP=0, TN=700, FN=6
Sensitivity: 0.98
Specificity: 1.0
Accuracy: 0.994
Balanced Accuracy: 0.99
Precision: 1.0
F1-Score: 0.99
MCC: 0.986

Predicted Performance of Digit Label/Class: 2
TP=400, FP=0, TN=600, FN=0
Sensitivity: 1.0
Specificity: 1.0
Accuracy: 1.0
Balanced Accuracy: 1.0
Precision: 1.0
F1-Score: 1.0
MCC: 1.0



In [18]:
##########################################################
# 8. OVERALL - FINAL PREDICTION PERFORMANCE
##########################################################
print("Overall Performance Prediction:");
print("Sensitivity: {0}%".format(round(mean(store_sens)*100, 4)));
print("Specificity: {0}%".format(round(mean(store_spec)*100, 4)));
print("Accuracy: {0}%".format(round(mean(store_acc)*100, 4)));
print("Balanced Accuracy: {0}%".format(round(mean(store_bal_acc)*100, 4)));
print("Precision: {0}%".format(round(mean(store_prec)*100, 4)));
print("F1-Score: {0}%".format(round(mean(store_fscore)*100, 4)))
print("MCC: {0}\n".format(round(mean(store_mcc), 4)))

Overall Performance Prediction:
Sensitivity: 99.3333%
Specificity: 99.7%
Accuracy: 99.6%
Balanced Accuracy: 99.5333%
Precision: 99.3333%
F1-Score: 99.3333%
MCC: 0.9907



In [19]:
pred_proba = ensemble.predict_proba(x_test); # predict probability for all target labels
pred_proba

array([[0.01821276, 0.87698798, 0.10479926],
       [0.00462802, 0.81744881, 0.17792318],
       [0.0056667 , 0.95534452, 0.03898878],
       ...,
       [0.98999479, 0.00333854, 0.00666667],
       [0.97864599, 0.01468734, 0.00666667],
       [0.97491055, 0.02175612, 0.00333334]])

In [20]:
pred_total =pred_proba[0,0] + pred_proba[0,1] + pred_proba[0,2]
pred_total2 =pred_proba[1,0] + pred_proba[1,1] + pred_proba[1,2]
pred_total3 =(pred_proba[2,0] + pred_proba[2,1] + pred_proba[2,2])


pred_total2

1.0

In [21]:
import time
start_time = time.time()
print("Time : %s seconds." % (time.time() - start_time))

Time : 5.4836273193359375e-05 seconds.
